# Standard BM25

Using the classic Cranfield dataset, this notebook shows how to use BM25 to calculate the similarity scores between a query and the documents and show the evaluation scores, i.e., precision and recall. Note that the ranking of the returned documents is not yet considered.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# load data into dataframes
docs = pd.read_json("data/cranfield_docs.json")
queries = pd.read_json("data/cranfield_queries.json")
relevance = pd.read_json("data/cranfield_relevance.json")

In [3]:
docs.head()

,author,bibliography,body,id,title
0,"brenckman,m.","j. ae. scs. 25, 1958, 324.",experimental investigation of the aerodynamics...,1,experimental investigation of the aerodynamics...
1,ting-yili,"department of aeronautical engineering, rensse...",simple shear flow past a flat plate in an inco...,2,simple shear flow past a flat plate in an inco...
2,m. b. glauert,"department of mathematics, university of manch...",the boundary layer in simple shear flow past a...,3,the boundary layer in simple shear flow past a...
3,"yen,k.t.","j. ae. scs. 22, 1955, 728.",approximate solutions of the incompressible la...,4,approximate solutions of the incompressible la...
4,"wasserman,b.","j. ae. scs. 24, 1957, 924.",one-dimensional transient heat conduction into...,5,one-dimensional transient heat conduction into...


In [4]:
queries.head()

,query,query_id
0,what similarity laws must be obeyed when const...,1
1,what are the structural and aeroelastic proble...,2
2,what problems of heat conduction in composite ...,3
3,can a criterion be developed to show empirical...,4
4,what chemical kinetic system is applicable to ...,5


In [5]:
queries.set_index('query_id', inplace=True)
queries.head()

,query
query_id,
1,what similarity laws must be obeyed when const...
2,what are the structural and aeroelastic proble...
3,what problems of heat conduction in composite ...
4,can a criterion be developed to show empirical...
5,what chemical kinetic system is applicable to ...


In [6]:
queries['query'][1]

'what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft .'

In [7]:
relevance.head()

,doc_id,query_id,r_score
0,184,1,2
1,29,1,2
2,31,1,2
3,12,1,3
4,51,1,3


In [8]:
set(relevance['r_score'].values)

{1, 2, 3, 4}

In [9]:
train_docs = docs['body'].tolist()
print(len(train_docs))
train_docs[:1]

1400


['experimental investigation of the aerodynamics of a wing in a slipstream .   an experimental study of a wing in a propeller slipstream was made in order to determine the spanwise distribution of the lift increase due to slipstream at different angles of attack of the wing and at different free stream to slipstream velocity ratios .  the results were intended in part as an evaluation basis for different theoretical treatments of this problem .   the comparative span loading curves, together with supporting evidence, showed that a substantial part of the lift increment produced by the slipstream was due to a /destalling/ or boundary-layer-control effect .  the integrated remaining lift increment, after subtracting this destalling lift, was found to agree well with a potential flow theory .   an empirical evaluation of the destalling effects was made for the specific configuration of the experiment .']

In [10]:
# https://tartarus.org/martin/PorterStemmer/def.txt
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
import re # regular expression


def stemming_tokenizer(str_input):
    words = re.sub(r"[^A-Za-z\-]", " ", str_input).lower().split() # delete non letter charactors
    #words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split() # include numbers
    words = [porter_stemmer.stem(word) for word in words]
    return words

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(tokenizer=stemming_tokenizer, max_df=0.9, min_df=0.1, stop_words='english', ngram_range=(1, 3))
train_docs = vectorizer.fit_transform(train_docs)
train_docs = vectorizer.inverse_transform(train_docs)

/Users/tezign/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words

In [12]:
import math
from six import iteritems
from six.moves import xrange

# BM25 parameters.
PARAM_K1 = 2.0
PARAM_B = 1.0
EPSILON = 0.25


class BM25(object):

    def __init__(self, corpus):
        self.corpus_size = len(corpus)
        self.avgdl = sum(map(lambda x: float(len(x)), corpus)) / self.corpus_size
        self.corpus = corpus
        self.f = []
        self.df = {}
        self.idf = {}
        self.initialize()
        self.average_idf = self.get_average_idf()

    def get_average_idf(self):
        return sum(map(lambda k: float(self.idf[k]), self.idf.keys())) / len(self.idf.keys())

    def initialize(self):
        for document in self.corpus:
            frequencies = {}
            for word in document:
                if word not in frequencies:
                    frequencies[word] = 0
                frequencies[word] += 1
            self.f.append(frequencies)

            for word, freq in iteritems(frequencies):
                if word not in self.df:
                    self.df[word] = 0
                self.df[word] += 1

        for word, freq in iteritems(self.df):
            self.idf[word] = math.log(self.corpus_size - freq + 0.5) - math.log(freq + 0.5)

    def get_score(self, document, index, average_idf):
        score = 0
        for word in document:
            if word not in self.f[index]:
                continue
            idf = self.idf[word] if self.idf[word] >= 0 else EPSILON * average_idf
            score += (idf * self.f[index][word] * (PARAM_K1 + 1)
                      / (self.f[index][word] + PARAM_K1 * (1 - PARAM_B + PARAM_B * self.corpus_size / self.avgdl)))
        return score

    def get_scores(self, document):
        scores = []
        for index in xrange(self.corpus_size):
            score = self.get_score(document, index, self.average_idf)
            scores.append(score)
        return scores

In [13]:
bm25 = BM25(train_docs)

In [14]:
def get_doc_ids(query_id, similarity_threshold):
#     query = queries['query'][query_id]
#     query = vectorizer.transform([query])
#     query = vectorizer.inverse_transform(query)
    query = stemming_tokenizer(queries['query'][query_id])
    print(query)
    test_result = bm25.get_scores(query)
    max_score = max(test_result) if max(test_result) > 0 else 1
    test_result = [x / max_score for i, x in enumerate(test_result)]
    # pd.DataFrame(test_result.toarray())
    df = pd.DataFrame(test_result)  # calculate the pair-wise similarity and convert to df
    df = df.rename(columns={0: 'similarity'})  # rename the first column
    df = df.sort_values('similarity', ascending=False)  # sort the result based on similarity score
    df_filtered = df[df['similarity'] > similarity_threshold]  # filter the result based on similarity threshold
    returned_doc_ids_list = df_filtered.index.tolist()  # get the ids of the returned docs
    return returned_doc_ids_list

In [15]:
# get the doc ids with relevance score
# this is the ground truth of relevance docs for the query based on the human annotated data
def get_true_doc_ids(query_id, relevance_threshold):
    # filter based on r_score (1, 2, 3, or 4) and relevance_threshold
    true_doc_ids = relevance[(relevance['query_id'] == query_id) & (relevance['r_score'] <= relevance_threshold)]
    true_doc_ids_list = true_doc_ids['doc_id'].to_list()
    return true_doc_ids_list

In [16]:
# calculate evaluation scores: precision and recall

def show_eval_scores(returned_doc_ids_list, true_doc_ids_list):
    # true positive
    tp = [x for x in true_doc_ids_list if x in returned_doc_ids_list]
    # tp.sort()
    # print(tp, len(tp))

    # false positive
    fp = [x for x in returned_doc_ids_list if x not in tp]
    # fp.sort()
    # print(fp, len(fp))

    # false negative
    fn = [x for x in true_doc_ids_list if x not in tp]
    # fn.sort()
    # print(fn, len(fn))

    precision = len(tp) / (len(tp) + len(fp))
    recall = len(tp) / (len(tp) + len(fn))

    print(f'precision is {precision:.3%}')
    print(f'recall is {recall:.3%}')

    return precision, recall

In [17]:
# utility function to put everything together
def show_result(query_id, similarity_threshold, relevance_threshold):
    print(f"query: {queries['query'][query_id]}")
    print('calculating results......')
    # we set a similarity threshold and get the ids of those documents
    # similarity_threshold 0.65 is used in https://www.aaai.org/Papers/FLAIRS/2003/Flairs03-082.pdf
    returned_doc_ids_list = get_doc_ids(query_id, similarity_threshold)

    # we choose relevance_threshold = 3, relevance 1, 2 and 3 mean relevant for returned documents
    # see readme for definitions about r_score
    true_doc_ids_list = get_true_doc_ids(query_id, relevance_threshold)

    print(f'similarity_threshold is {similarity_threshold} and relevance_threshold is {relevance_threshold}')

    show_eval_scores(returned_doc_ids_list, true_doc_ids_list)

In [18]:
# query_id = 1, similarity_threshold = 0.3, relevance_threshold = 3
show_result(1, 0.3, 3)

query: what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft .
calculating results......
['what', 'similar', 'law', 'must', 'be', 'obey', 'when', 'construct', 'aeroelast', 'model', 'of', 'heat', 'high', 'speed', 'aircraft']
similarity_threshold is 0.3 and relevance_threshold is 3
precision is 3.673%
recall is 40.909%


In [19]:
# query_id = 2, similarity_threshold = 0.3, relevance_threshold = 3
show_result(2, 0.3, 3)

query: what are the structural and aeroelastic problems associated with flight of high speed aircraft .
calculating results......
['what', 'are', 'the', 'structur', 'and', 'aeroelast', 'problem', 'associ', 'with', 'flight', 'of', 'high', 'speed', 'aircraft']
similarity_threshold is 0.3 and relevance_threshold is 3
precision is 1.705%
recall is 28.571%
